In [1]:
%pip install bs4


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install lxml


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pandas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests

url = "https://phy.iiserb.ac.in/faculty"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers, verify=False)
html = response.text

print(html[:500])

/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'phy.iiserb.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1">
<title>IISER Bhopal</title>

<!-- CSS -->
<link rel="stylesheet" href="http://fonts.googleapis.com/css?family=Open+Sans:400italic,400">
<link rel="stylesheet" href="http://fonts.googleapis.com/css?family=Droid+Sans">
<link rel="stylesheet" href="http://fonts.googleapis.com/css?family=Lobster">
<link r


In [6]:
# --- Imports ---
import requests
from bs4 import BeautifulSoup
import csv
import time
from urllib.parse import urljoin

requests.packages.urllib3.disable_warnings()

BASE_URL = "https://phy.iiserb.ac.in"
FACULTY_PAGE = f"{BASE_URL}/faculty"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Fetch faculty listing
response = requests.get(FACULTY_PAGE, headers=HEADERS, verify=False)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")

csv_file = "iiserb_physics_faculty_full.csv"
with open(csv_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow([
        "Section", "Name", "Designation", "Research Area",
        "Phone", "Email", "Image", "Profile Link", "PhD", "Postdoc"
    ])

    sections = soup.select("div.container > div.row")
    current_section = ""

    for section in sections:
        title_tag = section.select_one(".portfolio-filters a.filter-all")
        if title_tag:
            current_section = title_tag.get_text(strip=True)

        for faculty in section.select("div.col-sm-3"):
            try:
                img_tag = faculty.find("img")
                image = urljoin(BASE_URL, img_tag["src"]) if img_tag else ""

                name_tag = faculty.find("h3")
                name = name_tag.get_text(strip=True) if name_tag else ""
                profile_tag = name_tag.find("a") if name_tag else None
                profile_link = urljoin(BASE_URL, profile_tag["href"].strip()) if profile_tag else ""

                p_tag = faculty.find("p")
                designation = research = phone = email = ""
                if p_tag:
                    lines = [line.strip() for line in p_tag.decode_contents().split("<br>") if line.strip()]
                    designation = lines[0] if len(lines) > 0 else ""
                    research = lines[1] if len(lines) > 1 else ""
                    phone = lines[2] if len(lines) > 2 else ""
                    email = lines[3] if len(lines) > 3 else ""

                phd_institute = ""
                postdoc_institute = ""

                # Fetch profile details safely
                if profile_link:
                    try:
                        prof_resp = requests.get(profile_link, headers=HEADERS, verify=False, timeout=10)
                        prof_resp.raise_for_status()
                        prof_soup = BeautifulSoup(prof_resp.text, "html.parser")

                        academic_section = prof_soup.select_one("div.col-sm-9 ul")
                        if academic_section:
                            for li in academic_section.select("li"):
                                text = li.get_text(strip=True)
                                if ("Ph.D" in text or "PhD" in text) and not phd_institute:
                                    if "Department" in text:
                                        phd_institute = text.split("Department")[-1].strip()
                                    elif "from" in text:
                                        phd_institute = text.split("from")[-1].strip()
                                    else:
                                        phd_institute = text
                                elif ("Post-Doctoral" in text or "Postdoctoral" in text) and not postdoc_institute:
                                    if "Post-Doctoral Research Scholar" in text:
                                        postdoc_institute = text.split("Post-Doctoral Research Scholar")[-1].strip()
                                    elif "Postdoctoral" in text:
                                        postdoc_institute = text.split("Postdoctoral")[-1].strip()
                                    else:
                                        postdoc_institute = text

                        time.sleep(0.5)  # polite delay

                    except Exception as e:
                        print(f"⚠️ Profile fetch failed for {name} ({profile_link}): {e}")

                writer.writerow([
                    current_section, name, designation, research,
                    phone, email, image, profile_link,
                    phd_institute, postdoc_institute
                ])
                print(f"✅ Saved: {name}")

            except Exception as e:
                print(f"⚠️ Skipped faculty due to error: {e}")

print(f"🎉 Scraping completed! Data saved to {csv_file}")


✅ Saved: Arnab Rudra
✅ Saved: Mayuresh Surnis
✅ Saved: Nabamita Banerjee
✅ Saved: Rahul Srivastava
✅ Saved: Rajib Saha
✅ Saved: Ritam Mallick
✅ Saved: Sukanta Panda
✅ Saved: Suvankar Dutta
✅ Saved: Arnab Khan
✅ Saved: Dhanvir Singh Rana
✅ Saved: K V Adarsh
✅ Saved: Phani Kumar Peddibhotla
✅ Saved: Ravi Prakash Singh
⚠️ Profile fetch failed for Ravi Shankar Singh (https://phy.iiserb.ac.in/faculty_details?rssingh): 500 Server Error: Internal Server Error for url: https://phy.iiserb.ac.in/faculty_details?rssingh
✅ Saved: Ravi Shankar Singh
✅ Saved: Rohan Singh
✅ Saved: Surajit Saha
✅ Saved: Chandan Samanta
✅ Saved: Adarsh Bhaskar Vasista
✅ Saved: Auditya Sharma
✅ Saved: Nirmal Ganguli
✅ Saved: Snigdha Thakur
✅ Saved: Suhas Gangadharaiah
✅ Saved: Sunil Pratap Singh
⚠️ Profile fetch failed for Subhash Chaturvedi (https://phy.iiserb.ac.in/faculty_details?subhash): 500 Server Error: Internal Server Error for url: https://phy.iiserb.ac.in/faculty_details?subhash
✅ Saved: Subhash Chaturvedi
⚠️ 